In [1]:
from transformers.integrations import is_tensorboard_available
print(is_tensorboard_available())
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")
import torch
print(torch.cuda.get_device_capability())

True
CUDA available: True
Device name: NVIDIA GeForce RTX 3050
(8, 6)


In [5]:
# Split the dataset into train and eval
split_datasets = tokenized_dataset.train_test_split(test_size=0.1)  # Use 10% for evaluation
train_dataset = split_datasets["train"]
eval_dataset = split_datasets["test"]

# Update the Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Add eval dataset here
    tokenizer=tokenizer,
)

NameError: name 'tokenized_dataset' is not defined

In [10]:
from transformers.integrations import is_tensorboard_available
print(is_tensorboard_available())
import torch
print("CUDA available:", torch.cuda.is_available())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")
import torch
print(torch.cuda.get_device_capability())


# pip install tensorboard tqdm
from transformers import TrainingArguments
import os
from transformers import GPTJForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments, ProgressCallback
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from tqdm import tqdm  # For custom progress bars
import torch

# Step 1: Load all .txt files from the folder with a progress bar
def load_txt_files(folder_path):
    data = []
    for file_name in tqdm(os.listdir(folder_path), desc="Loading Text Files"):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as file:
                data.append({"text": file.read()})
    return data

def tokenize_data(tokenizer, dataset):
    def tokenize_function(examples):
        inputs = tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=1024,
        )
        return {"input_ids": inputs["input_ids"], "attention_mask": inputs["attention_mask"]}
    
    return dataset.map(tokenize_function, batched=True, desc="Tokenizing Data")

# Step 3: Prepare the dataset
folder_path = r"E:\downloads\txt_output"  # Path to the folder containing .txt files
data = load_txt_files(folder_path)
dataset = Dataset.from_list(data)  # Convert to Hugging Face Dataset

# Load tokenizer and tokenize the dataset
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token  # GPT-J doesn't have a pad token, so use EOS
tokenized_dataset = tokenize_data(tokenizer, dataset)


from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments

# Load model on CPU
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,  # Smallest batch size
    gradient_accumulation_steps=16,  # Simulate larger batch size
    fp16=False,  # Disable mixed precision on CPU
    num_train_epochs=3,
    learning_rate=5e-5,
    save_steps=1000,
    logging_steps=100,
    logging_dir="./logs",
    report_to="tensorboard",
)

from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Directly return the model outputs without computing loss
        outputs = model(**inputs)
        return (outputs, outputs) if return_outputs else outputs
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,  # Keep tokenizer for preprocessing
)


# Train the model
trainer.train()

# Save the model and tokenizer
trainer.save_model("./fine_tuned_gptneo_125m")
tokenizer.save_pretrained("./fine_tuned_gptneo_125m")


True
CUDA available: True
Device name: NVIDIA GeForce RTX 3050
(8, 6)


Loading Text Files: 100%|████████████████████████████████████████████████████████████| 52/52 [00:00<00:00, 4001.83it/s]


Tokenizing Data:   0%|          | 0/52 [00:00<?, ? examples/s]

C:\Users\esteb\AppData\Local\Temp\ipykernel_28844\2189832020.py:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


TypeError: unsupported operand type(s) for /: 'CausalLMOutputWithPast' and 'int'

In [11]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
from tqdm import tqdm
import os

# Step 1: Load all .txt files from the folder
def load_txt_files(folder_path):
    data = []
    for file_name in tqdm(os.listdir(folder_path), desc="Loading Text Files"):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, "r", encoding="utf-8") as file:
                data.append({"text": file.read()})
    return data

# Step 2: Tokenize the data and add labels
def tokenize_data(tokenizer, dataset):
    def tokenize_function(examples):
        inputs = tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=1024,
        )
        inputs["labels"] = inputs["input_ids"].copy()  # Add labels for loss calculation
        return inputs

    return dataset.map(tokenize_function, batched=True, desc="Tokenizing Data")

# Step 3: Load dataset
folder_path = r"E:\downloads\txt_output"  # Adjust to your dataset folder path
data = load_txt_files(folder_path)
dataset = Dataset.from_list(data)  # Convert list to Hugging Face Dataset

# Step 4: Load tokenizer and tokenize dataset
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set
tokenized_dataset = tokenize_data(tokenizer, dataset)

# Step 5: Load model
model = GPTNeoForCausalLM


Loading Text Files: 100%|████████████████████████████████████████████████████████████| 52/52 [00:00<00:00, 3059.35it/s]


Tokenizing Data:   0%|          | 0/52 [00:00<?, ? examples/s]

In [ ]:
#Use

In [ ]:
#pip install transformers torch
from transformers import GPTJForCausalLM, GPT2Tokenizer
import torch

# Path to the fine-tuned model and tokenizer
model_path = "./fine_tuned_gptj"

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_path)

# Load the fine-tuned model
model = GPTJForCausalLM.from_pretrained(model_path, torch_dtype=torch.float16).cuda()  # Use .cpu() if no GPU


In [ ]:
def generate_text(prompt, max_length=100, temperature=1.0, top_p=0.9):
    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Use .to("cpu") if no GPU
    
    # Generate text
    outputs = model.generate(
        inputs["input_ids"],
        max_length=max_length,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
        eos_token_id=tokenizer.eos_token_id
    )
    
    # Decode the generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example usage
prompt = "Once upon a time"
generated_text = generate_text(prompt)
print(generated_text)


In [ ]:
#pip install fastapi uvicorn

from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Prompt(BaseModel):
    text: str

@app.post("/generate")
def generate(prompt: Prompt):
    generated_text = generate_text(prompt.text)
    return {"response": generated_text}

# Run with: uvicorn app:app --reload


In [6]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Path to the folder containing the model and tokenizer
model_folder = r"E:\downloads\fine_tuned_gptneo_125m"

# Step 1: Load the Model and Tokenizer
def load_model_and_tokenizer(folder_path):
    # Ensure the folder contains all necessary files
    required_files = ["config.json", "pytorch_model.bin", "tokenizer.json", "vocab.json", "merges.txt"]
    if not all(os.path.isfile(os.path.join(folder_path, file)) for file in required_files):
        raise ValueError("The specified folder is missing required model or tokenizer files.")

    tokenizer = AutoTokenizer.from_pretrained(folder_path)
    model = AutoModelForCausalLM.from_pretrained(folder_path)
    model.eval()  # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Model and tokenizer successfully loaded on {device}.")
    return tokenizer, model, device

# Step 2: Question-Answer Interface
def interactive_qa(tokenizer, model, device):
    print("Interactive Q&A Interface")
    print("Type 'exit' to quit.")
    
    while True:
        # Get user input
        question = input("\nAsk a question: ")
        if question.lower() == "exit":
            print("Exiting interactive Q&A. Goodbye!")
            break

        try:
            max_tokens = int(input("Enter the maximum number of tokens for the answer (e.g., 50): "))
        except ValueError:
            print("Invalid input for max tokens. Using default of 50.")
            max_tokens = 50

        # Generate the response
        inputs = tokenizer(question, return_tensors="pt").to(device)
        output = model.generate(
            **inputs,
            max_length=inputs["input_ids"].shape[1] + max_tokens,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_k=50
        )
        answer = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f"\nAnswer: {answer}")

# Step 3: Main Function to Load Model and Start Interface
def main():
    try:
        tokenizer, model, device = load_model_and_tokenizer(model_folder)
        interactive_qa(tokenizer, model, device)
    except Exception as e:
        print(f"Error: {e}")

# Run the program
if __name__ == "__main__":
    main()


Error: The specified folder is missing required model or tokenizer files.


In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Path to the folder containing the model and tokenizer
#model_folder = r"E:\downloads\fine_tuned_gptneo_125m"
model_folder = r"fine_tuned_gptneo_125m"

# Step 1: Load the Model and Tokenizer
def load_model_and_tokenizer(folder_path):
    # Ensure the folder contains all necessary files
    required_files = ["config.json", "model.safetensors", "tokenizer.json", "vocab.json", "merges.txt"]
    if not all(os.path.isfile(os.path.join(folder_path, file)) for file in required_files):
        raise ValueError("The specified folder is missing required model or tokenizer files.")

    tokenizer = AutoTokenizer.from_pretrained(folder_path)
    model = AutoModelForCausalLM.from_pretrained(folder_path, trust_remote_code=True)
    model.eval()  # Set model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    print(f"Model and tokenizer successfully loaded on {device}.")
    return tokenizer, model, device

# Step 2: Question-Answer Interface
def interactive_qa(tokenizer, model, device):
    print("Interactive Q&A Interface")
    print("Type 'exit' to quit.")
    
    while True:
        # Get user input
        question = input("\nAsk a question: ")
        if question.lower() == "exit":
            print("Exiting interactive Q&A. Goodbye!")
            break

        try:
            max_tokens = 250 #int(input("Enter the maximum number of tokens for the answer (e.g., 50): "))
        except ValueError:
            print("Invalid input for max tokens. Using default of 50.")
            max_tokens = 50

        # Generate the response
        inputs = tokenizer(question, return_tensors="pt").to(device)
        output = model.generate(
            **inputs,
            max_length=inputs["input_ids"].shape[1] + max_tokens,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.7,
            top_k=50
        )
        answer = tokenizer.decode(output[0], skip_special_tokens=True)
        print(f"\nAnswer: {answer}")

# Step 3: Main Function to Load Model and Start Interface
def main():
    try:
        tokenizer, model, device = load_model_and_tokenizer(model_folder)
        interactive_qa(tokenizer, model, device)
    except Exception as e:
        print(f"Error: {e}")

# Run the program
if __name__ == "__main__":
    main()



Model and tokenizer successfully loaded on cuda.
Interactive Q&A Interface
Type 'exit' to quit.



Ask a question:  cicima ucr give me paper article name



Answer: cicima ucr give me paper article name on the list and link for it
<mrooney> ok
<mrooney> ok cool
<mrooney>  /join #ubuntu-it-offtopic
<mrooney> https://www.tuxfiles.org/
<mrooney>  /join #ubuntu-offtopic
<mrooney>  /join #ubuntu-offtopic
<mrooney>  /join #ubuntu-offtopic
<mrooney>  /join #ubuntu-offtopic
<mrooney> https://www.tuxfiles.org/
<mrooney>  /join #ubuntu-offtopic
<mrooney> https://www.tuxfiles.org/
<hanswers> Hi
<mrooney> https://www.tuxfiles.org/
<hanswers> Hi
<hanswers> Hi
<hanswers> How can I enable the new initscript in the XF86?
<hanswers>  I don't see a way to make a new initscript, is there?
<hanswers>



Ask a question:  give me the names of every article with material science in its name



Answer: give me the names of every article with material science in its name.

The authors of this publication provide full text of their research. The authors are not responsible for the content of any content on this page.

The content on this page may not be reproduced, published, displayed, or referred to in any way without prior author's written permission in accordance with the terms of the Licence.
